In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_autoinstaller
import pandas as pd
import re

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')

import spacy
nlp = spacy.load("en_core_web_sm")




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Emre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

game_id = "413150"
steam_url = f"https://steamcommunity.com/app/{game_id}/reviews/?filterLanguage=english&p=1&browsefilter=toprated"
driver.get(steam_url)
driver.maximize_window()
time.sleep(5) #yükleme için bekleme
start_time = time.time() 

## Datayı toplama


In [3]:
def scroll_the_website():

    while True:   

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        if time.time()-start_time > 200:
            break
        

In [4]:
scroll_the_website()

reviews = driver.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')
review_list = []
def clean_the_data():
    for review in reviews:
        
        review_text = review.text
        review_text = review_text.split("\n")[1:] #yayınlanma tarihi skipleme    
        review_text = [item.replace("Ürün ücretsiz alındı","") for item in review_text] 
        review_text = ' '.join(review_text)
        review_text = re.sub('[^a-zA-Z]',' ',review_text)
        review_text = review_text.lower()
        doc = nlp(review_text) 
        review_text = " ".join([token.lemma_ for token in doc]) #lemmatization
        review_list.append(review_text)

clean_the_data()
driver.quit()

In [5]:
df_data = pd.DataFrame(review_list)
df_data.head(len(df_data))
df_data.to_csv('sentiment_data.csv', index=False)

In [6]:
tokens = [word_tokenize(data) for data in review_list]
df_tokens = pd.DataFrame(tokens)
df_tokens.head(len(df_tokens))

,0,1,2,3,4,5,6,7,8,9,...,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561
0,love,be,able,to,sit,back,and,relax,while,I,...,None,None,None,None,None,None,None,None,None,None
1,brilliant,fantastic,farming,sim,very,accurate,as,I,can,tell,...,None,None,None,None,None,None,None,None,None,None
2,this,be,a,very,good,game,I,have,a,hard,...,None,None,None,None,None,None,None,None,None,None
3,refund,bf,for,this,way,well,choice,and,money,well,...,None,None,None,None,None,None,None,None,None,None
4,help,I,fight,my,porn,addiction,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,its,and,the,rear,view,mirror,still,do,not,work,...,None,None,None,None,None,None,None,None,None,None
1006,cool,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1007,I,enjoy,the,full,immersion,that,farming,bring,I,try,...,None,None,None,None,None,None,None,None,None,None
1008,in,the,morning,I,can,see,and,join,my,friend,...,None,None,None,None,None,None,None,None,None,None


In [7]:
tagged_reviews = [nltk.pos_tag(review) for review in tokens]


